## Huggingface transformers basics 

### customizing pretrained model

General scheme for leveraging a pretrained model on a downstream task is Loading a model weight and modifying its architecture.

허깅페이스 허브에서는 모델의 사전훈련 가중치를 다운받을 수 있고,  
허깅페이스 라이브러리로 훈련 및 추론을 할 때는 모델 선택 등 많은 부분을 자동화되어 있어 편리하다.    
  
하지만 모델 설계 변경,
classification task 등을 위해 classification head 부착 후 훈련 등,  
모델 가중치 로드 뿐 아니라 모델을 직접 변경하여야 할 때가 있다.
  
목표: RoBERTa 모델에 허깅페이스 모델 체크포인트(훈련된 가중치)를 로드 뒤  
모델 일부 (뒷부분 헤드)을 변경한다.

### Load model

transformers의 AutoModel은 모델 체크포인트가 훈련되었던 모델 구조를 편리하게 로드한다.  
체크포인트로부터 모델을 로드하면 바로 DebertaV2Model 오브젝트가 로드된다.  
해당 오브젝트의 doc 설명(DebertaV2Model)의 설명에 따르면  
DebertaV2Model은 추가 헤드 없이 raw hidden-state를 출력하는 DeBERTa 구조이다.




In [1]:
import transformers
from transformers import AutoModel

model = AutoModel.from_pretrained("google-bert/bert-base-cased")
print(model)
model = AutoModel.from_pretrained("microsoft/deberta-v3-small")
print(model)
# for additional detail usage > [doc]auto https://huggingface.co/docs/transformers/model_doc/auto

from transformers import AutoConfig, AutoModel
NewModelConfig = AutoConfig.from_pretrained("google-bert/bert-base-cased")
print(NewModelConfig)

# AutoConfig은 "microsoft/deberta-v3-small" checkpoint에 맞는 DebertaV2Config 오브젝트를 초기화하고 해당 Config을 자동으로 로드함
NewModelConfig = AutoConfig.from_pretrained("microsoft/deberta-v3-small")
print(NewModelConfig) 

# DebertaV2Config 오브젝트를 명시적으로 지정후 체크포인트 명시해서 로드시 같은 Config으로 되어 있음
from transformers import DebertaV2Config
NewDebertaV2ModelConfig = DebertaV2Config.from_pretrained("microsoft/deberta-v3-small")
print(NewDebertaV2ModelConfig) 

'''In detail information of DebertaV2Config and   
different objects list(model/tokenizer/etc) of Huggingface DeBERTa-v2 model   
that is applicable for different learning scenario >   
[doc]DeBERTa-v2 https://huggingface.co/docs/transformers/model_doc/deberta-v2#transformers.DebertaV2Config'''

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

/usr/local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(128100, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
    (dropout): StableDropout()
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0-5): 6 x DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=768, out_features=768, bias=True)
            (key_proj): Linear(in_features=768, out_features=768, bias=True)
            (value_proj): Linear(in_features=768, out_features=768, bias=True)
            (pos_dropout): StableDropout()
            (dropout): StableDropout()
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
        )
        (intermediate

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BertConfig {
  "_name_or_path": "google-bert/bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_

'In detail information of DebertaV2Config and   \ndifferent objects list(model/tokenizer/etc) of Huggingface DeBERTa-v2 model   \nthat is applicable for different learning scenario >   \n[doc]DeBERTa-v2 https://huggingface.co/docs/transformers/model_doc/deberta-v2#transformers.DebertaV2Config'

### Auto task model for a specific task

AutoModel는 모델 체크포인트 정보만 있으면 모델을 로드하지만 목적하는 태스크에 맞는 헤드 등 직접 작성해야 함  
자주 쓰는 태스크의 경우(e.g. sequence classification)
태스크에 맞는 Auto 클래스를 체크포인트와 함께 지정하면   
허깅페이스에서 미리 작성해둔 태스크용 모델을 자동 선택하기 때문에    
목적하는 태스크에 맞는 헤드 등을 직접 작성할 필요 없어 편리하게 이용 가능  
(e.g. sequence classification은 AutoModel 대신 AutoModelForSequenceClassification 이용)   


파이토치 모델이고 구조는 모두 공개되어 있으므로, 이 이후 모델 구조를 직접 자유롭게 고치는 것도 가능

사용법은 AutoModel과 동일  
"configuration" 지정하면 시작부터 태스크용 모델 훈련  
"체크포인트" 형식으로 지정하면 가중치가 적용된 태스크용 모델을 돌려줌  
체크포인트 경우 from_pretrained 를 이용해 체크포인트만 지정하면 됨  

(e.g. 시퀀스 데이터 분류 태스크의 경우 AutoModelForSequenceClassification을 임포트후  
 Debertav2 모델의 어떤 체크포인트를 불러오면 
Debertav2 모델의 sequence classification 모델을 따로 돌려줌)

모든 Auto 클래스는 문서 확인. 예시 문서 AutoModelForSequenceClassification
[doc] AutoModelForSequenceClassification : 
https://huggingface.co/docs/transformers/v4.41.2/en/model_doc/auto#transformers.AutoModelForSequenceClassification


In [2]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-small")
print(model)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-5): 6 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=T

###  AutoTokenizer를 사용한 토크나이저 불러오기
DebertaV2Tokenizer를 자동으로 로드한다.(이 노트북 스코프에서는 토크나이저를 자세히 다루지 않음)

In [3]:
from transformers import AutoTokenizer, AutoTokenizer

# https://discuss.huggingface.co/t/error-with-new-tokenizers-urgent/2847/3

# v4.0.0 이후 AutoTokenizer 및 파이프라인은 fast (rust) tokenizer를 기본값으로 사용함
# [github]huggingface transformers https://github.com/huggingface/transformers/releases/tag/v4.0.0
# newTokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")
# print(newTokenizer)
newTokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small", use_fast=False)
print(newTokenizer)


newTokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
print(newTokenizer)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

DebertaV2Tokenizer(name_or_path='microsoft/deberta-v3-small', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='google-bert/bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [10]:
text = "I am talking potato"

print(newTokenizer.encode(text))
print(newTokenizer.tokenize(text))

[101, 146, 1821, 2520, 17947, 102]
['I', 'am', 'talking', 'potato']
